In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### 1. Load files

In [3]:
FG_FILE = "C:\\Users\\harsh\\Downloads\\fear_greed_index.csv"
HIST_FILE = "C:\\Users\\harsh\\Downloads\\historical_data.csv"

fg = pd.read_csv(FG_FILE)
hist = pd.read_csv(HIST_FILE)

### 2. Normalize column names and show them

In [4]:
fg.columns = fg.columns.str.strip().str.lower()
hist.columns = hist.columns.str.strip().str.lower()

In [5]:
print("Fear&Greed columns:", list(fg.columns))
print("Historical columns:", list(hist.columns))

Fear&Greed columns: ['timestamp', 'value', 'classification', 'date']
Historical columns: ['account', 'coin', 'execution price', 'size tokens', 'size usd', 'side', 'timestamp ist', 'start position', 'direction', 'closed pnl', 'transaction hash', 'order id', 'crossed', 'fee', 'trade id', 'timestamp']


### 3. Auto-detect important columns

In [6]:
# Date columns
fg_date_col = next((c for c in fg.columns if "date" in c), None)
hist_date_col = next((c for c in hist.columns if "time" in c or "date" in c), None)


In [7]:
# Sentiment label (classification) and numeric index/value
classification_col = next((c for c in fg.columns if "class" in c or "classification" in c or "label" in c), None)
index_col = next((c for c in fg.columns if "value" in c or "index" in c or "score" in c), None)

In [8]:
# Closed PnL in historical
closedpnl_col = next((c for c in hist.columns if "closed" in c and "pnl" in c), None)
# Fallback: a column that contains 'pnl'
if closedpnl_col is None:
    closedpnl_col = next((c for c in hist.columns if "pnl" in c), None)


In [9]:
# Optional useful columns
leverage_col = next((c for c in hist.columns if "leverage" in c), None)
size_col = next((c for c in hist.columns if "size" in c), None)
account_col = next((c for c in hist.columns if "account" in c), None)
symbol_col = next((c for c in hist.columns if "symbol" in c), None)
side_col = next((c for c in hist.columns if "side" in c), None)

In [10]:
print("Detected columns:")
print(" fg_date_col =", fg_date_col)
print(" hist_date_col =", hist_date_col)
print(" classification_col =", classification_col)
print(" index_col =", index_col)
print(" closedpnl_col =", closedpnl_col)
print(" leverage_col =", leverage_col)
print(" size_col =", size_col)
print(" account_col =", account_col)
print(" symbol_col =", symbol_col)
print(" side_col =", side_col)

Detected columns:
 fg_date_col = date
 hist_date_col = timestamp ist
 classification_col = classification
 index_col = value
 closedpnl_col = closed pnl
 leverage_col = None
 size_col = size tokens
 account_col = account
 symbol_col = None
 side_col = side


### 4. Parse dates and create date-only fields

In [11]:
if fg_date_col:
    fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors="coerce")
    fg["date_only"] = fg[fg_date_col].dt.date
else:
    fg["date_only"] = pd.NaT

if hist_date_col:
    hist[hist_date_col] = pd.to_datetime(hist[hist_date_col], errors="coerce")
    hist["date_only"] = hist[hist_date_col].dt.date
else:
    hist["date_only"] = pd.NaT

### 5. Prepare historical numeric columns

In [12]:
if closedpnl_col:
    hist[closedpnl_col] = pd.to_numeric(hist[closedpnl_col], errors="coerce")
    hist["win"] = hist[closedpnl_col] > 0
else:
    hist["win"] = np.nan

if leverage_col:
    hist[leverage_col] = pd.to_numeric(hist[leverage_col], errors="coerce")
if size_col:
    hist[size_col] = pd.to_numeric(hist[size_col], errors="coerce")

### 6. Daily aggregation

In [13]:
agg_dict = {}
if closedpnl_col:
    agg_dict[closedpnl_col] = ['count','sum','mean','median','std']
agg_dict['win'] = ['mean']   # daily win rate
if leverage_col:
    agg_dict[leverage_col] = ['mean','median']
if size_col:
    agg_dict[size_col] = ['mean','median']

daily = hist.groupby('date_only').agg(agg_dict)

daily.columns = ['_'.join(filter(None, map(str,col))).strip() for col in daily.columns.values]
daily = daily.reset_index().rename(columns={'date_only':'date'})
print("Daily aggregated rows:", len(daily))

Daily aggregated rows: 480


### 7. Prepare fear/greed for merge

In [14]:
fg_for_merge = fg.copy()

if index_col:
    fg_for_merge[index_col] = pd.to_numeric(fg_for_merge[index_col], errors="coerce")

fg_for_merge = fg_for_merge[['date_only', classification_col, index_col] if index_col else ['date_only', classification_col]]
fg_for_merge = fg_for_merge.rename(columns={'date_only':'date'})


### 8. Merge daily with sentiment

In [15]:
merged = pd.merge(daily, fg_for_merge, on='date', how='left')

### 9. Create sentiment numeric if needed

In [16]:
if index_col:
    merged['sentiment_numeric'] = merged[index_col]
else:
    # Map ordinal labels to numeric if classification exists
    if classification_col:
        # common mapping; adjust if you prefer different scale
        mapping = {
            'extreme fear': 0,
            'fear': 1,
            'neutral': 2,
            'greed': 3,
            'extreme greed': 4
        }
        # lower-case mapping safe-guard
        merged['sent_lower'] = merged[classification_col].astype(str).str.strip().str.lower()
        merged['sentiment_numeric'] = merged['sent_lower'].map(mapping)
        merged = merged.drop(columns=['sent_lower'])
    else:
        merged['sentiment_numeric'] = np.nan

### 10. Correlations

In [17]:
corr_cols = []
if 'sentiment_numeric' in merged.columns:
    corr_cols.append('sentiment_numeric')
if closedpnl_col and f"{closedpnl_col}_mean" in merged.columns:
    corr_cols.append(f"{closedpnl_col}_mean")
if 'win_mean' in merged.columns:
    corr_cols.append('win_mean')

print("\nColumns used for correlation:", corr_cols)
if len(corr_cols) >= 2:
    pearson = merged[corr_cols].corr(method='pearson')
    spearman = merged[corr_cols].corr(method='spearman')
    print("\nPearson correlation:\n", pearson.round(4))
    print("\nSpearman correlation:\n", spearman.round(4))
else:
    print("Not enough columns to compute correlations.")



Columns used for correlation: ['sentiment_numeric', 'closed pnl_mean', 'win_mean']

Pearson correlation:
                    sentiment_numeric  closed pnl_mean  win_mean
sentiment_numeric             1.0000           -0.002    0.1021
closed pnl_mean              -0.0020            1.000    0.2400
win_mean                      0.1021            0.240    1.0000

Spearman correlation:
                    sentiment_numeric  closed pnl_mean  win_mean
sentiment_numeric             1.0000           0.0739    0.1356
closed pnl_mean               0.0739           1.0000    0.6671
win_mean                      0.1356           0.6671    1.0000


### 11. Quick value counts for classification

In [18]:
if classification_col:
    print("\nFear/Greed classification counts:")
    print(fg[classification_col].value_counts(dropna=False))


Fear/Greed classification counts:
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: classification, dtype: int64


### 12. Save merged csv

In [23]:
OUT_MERGED = "D:\\Primetrade.ai\\merged_daily_data.csv"
merged.to_csv(OUT_MERGED, index=False)
print("\nSaved merged daily data to:", OUT_MERGED)


Saved merged daily data to: D:\Primetrade.ai\merged_daily_data.csv


### 13. Plots

In [24]:
# 13A: Scatter: sentiment vs avg closed pnl
plt.figure(figsize=(8,5))
y_col = f"{closedpnl_col}_mean" if closedpnl_col else None
if 'sentiment_numeric' in merged.columns and y_col in merged.columns:
    plt.scatter(merged['sentiment_numeric'], merged[y_col], s=20)
    plt.xlabel("Sentiment (numeric)")
    plt.ylabel("Average Daily Closed PnL")
    plt.title("Sentiment vs Avg Closed PnL per Day")
    plt.grid(True)
    OUT_SCATTER = "D:\\Primetrade.ai\\plot_sentiment_vs_pnl.png"
    plt.tight_layout()
    plt.savefig(OUT_SCATTER)
    print("Saved scatter plot:", OUT_SCATTER)
    plt.close()
else:
    print("Skipping scatter plot (missing columns).")

Saved scatter plot: D:\Primetrade.ai\plot_sentiment_vs_pnl.png


In [25]:
# 13B: Time series plot of avg pnl and sentiment
if y_col in merged.columns and 'sentiment_numeric' in merged.columns:
    # ensure a datetime index
    merged['date_dt'] = pd.to_datetime(merged['date'])
    ts = merged.set_index('date_dt')[[y_col, 'sentiment_numeric']].dropna()
    if not ts.empty:
        ax = ts.plot(figsize=(12,5), subplots=False, legend=True)
        ax.set_title("Daily Avg Closed PnL and Sentiment")
        ax.set_xlabel("Date")
        OUT_TS = "D:\\Primetrade.ai\\time_series_pnl_sentiment.png"
        plt.tight_layout()
        plt.savefig(OUT_TS)
        print("Saved time-series plot:", OUT_TS)
        plt.close()
    else:
        print("Time series plot skipped (no non-null data).")


Saved time-series plot: D:\Primetrade.ai\time_series_pnl_sentiment.png


### 14. Optional: per-account summary file

In [26]:
if account_col:
    acct_agg = hist.groupby(account_col).agg({
        closedpnl_col: ['count','sum','mean','median'],
        'win': ['mean']
    })
    acct_agg.columns = ['_'.join(col).strip() for col in acct_agg.columns.values]
    acct_agg = acct_agg.reset_index().sort_values(by=f"{closedpnl_col}_sum", ascending=False)
    OUT_ACCT = "D:\\Primetrade.ai\\per_account_data.csv"
    acct_agg.to_csv(OUT_ACCT, index=False)
    print("Saved per-account data to:", OUT_ACCT)

print("\nDone.")


Saved per-account data to: D:\Primetrade.ai\per_account_data.csv

Done.
